In [1]:
import sys
sys.path.append("../")

In [5]:
import os
import hashlib
import numpy as np
import traceback
from common.dataloader import load_dataset
from matplotlib import pyplot as plt

In [18]:
def onehot2interval(arr):
    result = []
    record = False
    for idx, item in enumerate(arr):
        if item == 1 and not record:
            start = idx
            record = True
        if item == 0 and record:
            end = idx  # not include the end point, like [a,b)
            record = False
            result.append((start, end))
    return result

In [40]:
dataset = "SMD" # change to HUAWEI
subdatasets = ["machine-1-1"] # Change to skipped hashid list
outdir = f"./{dataset}_figures"
key = "test"
chunk_size = 50

for subdataset in subdatasets:
    current_out = os.path.join(outdir,subdataset) 
    os.makedirs(current_out, exist_ok=True)
    data_dict = load_dataset(dataset, subdataset, "all", root_dir="../")

    dims = list(range(data_dict[key].shape[1]))
    for cidx, chunk in enumerate(np.array_split(dims, 1 + len(dims)//chunk_size)):
        fig, ax = plt.subplots(len(chunk), figsize=(15, 4*len(chunk)))
        for idx, dim_idx in enumerate(chunk):
            data = data_dict[key][:, dim_idx]
            if key == "test":
                label = data_dict["test_labels"]
                intervals = onehot2interval(label)
                ax[idx].plot(data, label=f"{subdataset} : {dim_idx}")
                for s,e in intervals:
                    ax[idx].axvspan(s, e, alpha=0.2, color='red')
                ax[idx].legend()
        fig.savefig(os.path.join(current_out, f"chunk_{cidx}.pdf"))
        plt.close()
    print(f"Finish plotting {subdataset}")
    pass

Loading machine-1-1 of SMD dataset
.././datasets/anomaly/SMD/processed/machine-1-1_train.pkl
Shape of train is (28479, 38).
Shape of test is (28479, 38).
Shape of test_labels is (28479,).
Finish plotting machine-1-1
